# 모델 파일 및 레이아웃

확산 모델은 다양한 파일 유형으로 저장되고 다른 레이아웃으로 구성됩니다. Diffusers는 모델 가중치를 *Diffusers-다중 폴더* 레이아웃에 safetensors 파일로 저장하며, 확산 생태계에서 일반적으로 사용되는 *단일 파일* 레이아웃에서 파일(safetensors 및 ckpt 파일 등) 로드도 지원합니다.

각 레이아웃에는 고유한 이점과 사용 사례가 있으며, 이 가이드에서는 다른 파일 및 레이아웃을 로드하고 변환하는 방법을 보여줍니다.

## 파일

PyTorch 모델 가중치는 일반적으로 Python의 [pickle](https://docs.python.org/3/library/pickle.html) 유틸리티를 사용하여 ckpt 또는 bin 파일로 저장됩니다. 그러나 pickle은 안전하지 않으며 피클된 파일에는 실행될 수 있는 악성 코드가 포함될 수 있습니다. 이 취약점은 모델 공유의 인기를 감안할 때 심각한 문제입니다. 이 보안 문제를 해결하기 위해 [Safetensors](https://hf.co/docs/safetensors) 라이브러리가 pickle의 안전한 대안으로 개발되었으며 모델을 safetensors 파일로 저장합니다.

### safetensors

> [!TIP]
> 모델 가중치를 저장하고 로드하는 데 safetensor 파일이 선호되는 이유와 설계 결정에 대한 자세한 내용은 [Safetensors 감사 결과 정말 안전하며 기본값이 됨](https://blog.eleuther.ai/safetensors-security-audit/) 블로그 게시물을 참조하십시오.

[Safetensors](https://hf.co/docs/safetensors)는 텐서를 안전하고 빠르게 저장하고 로드하기 위한 안전하고 빠른 파일 형식입니다. Safetensors는 특정 유형의 공격을 제한하기 위해 헤더 크기를 제한하고, 지연 로딩(분산 설정에 유용함)을 지원하며, 일반적으로 로딩 속도가 더 빠릅니다.

[Safetensors](https://hf.co/docs/safetensors) 라이브러리가 설치되어 있는지 확인하십시오.

In [ ]:
!pip install safetensors

Safetensors는 가중치를 safetensors 파일에 저장합니다. Diffusers는 사용 가능하고 Safetensors 라이브러리가 설치되어 있는 경우 기본적으로 safetensors 파일을 로드합니다. safetensors 파일을 구성하는 방법에는 두 가지가 있습니다.

1. Diffusers-다중 폴더 레이아웃: 각 파이프라인 구성 요소(텍스트 인코더, UNet, VAE)에 대해 하나씩 여러 개의 개별 safetensors 파일이 있을 수 있으며, 하위 폴더로 구성됩니다(예: [stable-diffusion-v1-5/stable-diffusion-v1-5](https://hf.co/stable-diffusion-v1-5/stable-diffusion-v1-5/tree/main) 리포지토리 참조).
2. 단일 파일 레이아웃: 모든 모델 가중치가 단일 파일에 저장될 수 있습니다(예: [WarriorMama777/OrangeMixs](https://hf.co/WarriorMama777/OrangeMixs/tree/main/Models/AbyssOrangeMix) 리포지토리 참조).

<hfoptions id="safetensors">
<hfoption id="multifolder">

[from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained) 메서드를 사용하여 여러 폴더에 저장된 safetensors 파일이 있는 모델을 로드합니다.

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    use_safetensors=True
)

</hfoption>
<hfoption id="single file">

[from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드를 사용하여 모든 가중치가 단일 safetensors 파일에 저장된 모델을 로드합니다.

In [ ]:
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_single_file(
    "https://huggingface.co/WarriorMama777/OrangeMixs/blob/main/Models/AbyssOrangeMix/AbyssOrangeMix.safetensors"
)

</hfoption>
</hfoptions>

#### LoRA 파일

[LoRA](https://hf.co/docs/peft/conceptual_guides/adapter#low-rank-adaptation-lora)는 빠르고 쉽게 학습할 수 있는 경량 어댑터로, 특정 방식이나 스타일로 이미지를 생성하는 데 특히 인기가 있습니다. 이러한 어댑터는 일반적으로 safetensors 파일에 저장되며 [civitai](https://civitai.com/)와 같은 모델 공유 플랫폼에서 널리 사용됩니다.

LoRA는 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/loaders/lora#diffusers.loaders.StableDiffusionLoraLoaderMixin.load_lora_weights) 메서드를 사용하여 기본 모델에 로드됩니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

# 기본 모델
pipeline = StableDiffusionXLPipeline.from_pretrained(
    "Lykon/dreamshaper-xl-1-0", torch_dtype=torch.float16, variant="fp16"
).to("cuda")

# LoRA 가중치 다운로드
!wget https://civitai.com/api/download/models/168776 -O blueprintify.safetensors

# LoRA 가중치 로드
pipeline.load_lora_weights(".", weight_name="blueprintify.safetensors")
prompt = "bl3uprint, a highly detailed blueprint of the empire state building, explaining how to build all parts, many txt, blueprint grid backdrop"
negative_prompt = "lowres, cropped, worst quality, low quality, normal quality, artifacts, signature, watermark, username, blurry, more than one bridge, bad architecture"

image = pipeline(
    prompt=prompt,
    negative_prompt=negative_prompt,
    generator=torch.manual_seed(0),
).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/blueprint-lora.png"/>
</div>

### ckpt

> [!WARNING]
> 피클된 파일은 악성 코드를 실행하는 데 악용될 수 있으므로 안전하지 않을 수 있습니다. 가능한 경우 safetensors 파일을 대신 사용하거나 가중치를 safetensors 파일로 변환하는 것이 좋습니다.

PyTorch의 [torch.save](https://pytorch.org/docs/stable/generated/torch.save.html) 함수는 Python의 [pickle](https://docs.python.org/3/library/pickle.html) 유틸리티를 사용하여 모델을 직렬화하고 저장합니다. 이러한 파일은 ckpt 파일로 저장되며 전체 모델의 가중치를 포함합니다.

[from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드를 사용하여 ckpt 파일을 직접 로드합니다.

In [ ]:
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_single_file(
    "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/blob/main/v1-5-pruned.ckpt"
)

## 저장소 레이아웃

모델 파일은 Diffusers-다중 폴더 레이아웃 또는 단일 파일 레이아웃의 두 가지 방식으로 구성됩니다. Diffusers-다중 폴더 레이아웃이 기본값이며 각 구성 요소 파일(텍스트 인코더, UNet, VAE)은 별도의 하위 폴더에 저장됩니다. Diffusers는 모든 구성 요소가 함께 번들로 제공되는 단일 파일 레이아웃에서 모델 로드도 지원합니다.

### Diffusers-다중 폴더

Diffusers-다중 폴더 레이아웃은 Diffusers의 기본 저장소 레이아웃입니다. 각 구성 요소(텍스트 인코더, UNet, VAE)의 가중치는 별도의 하위 폴더에 저장됩니다. 가중치는 safetensors 또는 ckpt 파일로 저장할 수 있습니다.

<div class="flex flex-row gap-4">
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/multifolder-layout.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">다중 폴더 레이아웃</figcaption>
  </div>
  <div class="flex-1">
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/multifolder-unet.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">UNet 하위 폴더</figcaption>
  </div>
</div>

Diffusers-다중 폴더 레이아웃에서 로드하려면 [from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained) 메서드를 사용합니다.

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to("cuda")

Diffusers-다중 폴더 레이아웃 사용의 이점은 다음과 같습니다.

1. 각 구성 요소 파일을 개별적으로 또는 병렬로 더 빠르게 로드합니다.
2. 필요한 구성 요소만 로드하므로 메모리 사용량이 줄어듭니다. 예를 들어 [SDXL Turbo](https://hf.co/stabilityai/sdxl-turbo), [SDXL Lightning](https://hf.co/ByteDance/SDXL-Lightning) 및 [Hyper-SD](https://hf.co/ByteDance/Hyper-SD)와 같은 모델은 UNet을 제외하고 동일한 구성 요소를 갖습니다. [from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe) 메서드를 사용하여 공유 구성 요소를 재사용하고 추가 메모리를 소비하지 않고 UNet만 로드할 수 있습니다([파이프라인 재사용](https://huggingface.co/docs/diffusers/main/en/using-diffusers/./loading#reuse-a-pipeline) 가이드 참조). 이렇게 하면 중복 구성 요소를 다운로드하고 불필요하게 더 많은 메모리를 사용할 필요가 없습니다.

    ```py
    import torch
    from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler

    # 하나의 모델 다운로드
    sdxl_pipeline = StableDiffusionXLPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True,
    ).to("cuda")

    # 다른 모델에 대한 UNet 전환
    unet = UNet2DConditionModel.from_pretrained(
        "stabilityai/sdxl-turbo",
        subfolder="unet",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    )
    # UNet을 제외한 새 모델에서 동일한 모든 구성 요소 재사용
    turbo_pipeline = StableDiffusionXLPipeline.from_pipe(
        sdxl_pipeline, unet=unet,
    ).to("cuda")
    turbo_pipeline.scheduler = EulerDiscreteScheduler.from_config(
        turbo_pipeline.scheduler.config,
        timestep+spacing="trailing"
    )
    image = turbo_pipeline(
        "an astronaut riding a unicorn on mars",
        num_inference_steps=1,
        guidance_scale=0.0,
    ).images[0]
    image
    ```

3. SDXL [VAE](https://hf.co/madebyollin/sdxl-vae-fp16-fix)와 같이 여러 모델에서 구성 요소가 공유되는 경우 중복 다운로드 및 저장을 피하고 단일 복사본만 다운로드하고 저장하면 되므로 저장소 요구 사항이 줄어듭니다. SDXL 모델 10개의 경우 약 3.5GB의 저장 공간을 절약할 수 있습니다. PixArt Sigma와 같은 최신 모델의 경우 [텍스트 인코더](https://hf.co/PixArt-alpha/PixArt-Sigma-XL-2-1024-MS/tree/main/text_encoder)만으로도 약 19GB이므로 저장 공간 절약 효과는 더욱 커집니다!
4. 모델의 구성 요소를 더 새롭거나 더 나은 버전으로 교체할 수 있는 유연성입니다.

    ```py
    from diffusers import DiffusionPipeline, AutoencoderKL

    vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16, use_safetensors=True)
    pipeline = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        vae=vae,
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True,
    ).to("cuda")
    ```

5. 각 구성 요소 하위 폴더의 [config.json](https://hf.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/unet/config.json) 파일에 저장된 모델 구성 요소에 대한 가시성 및 정보 향상.

### 단일 파일

단일 파일 레이아웃은 모든 모델 가중치를 단일 파일에 저장합니다. 모든 모델 구성 요소(텍스트 인코더, UNet, VAE) 가중치는 하위 폴더에 별도로 저장되는 대신 함께 보관됩니다. 이는 safetensors 또는 ckpt 파일일 수 있습니다.

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/single-file-layout.png"/>
</div>

단일 파일 레이아웃에서 로드하려면 [from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드를 사용합니다.

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline

pipeline = StableDiffusionXLPipeline.from_single_file(
    "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0.safetensors",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to("cuda")

단일 파일 레이아웃 사용의 이점은 다음과 같습니다.

1. 일반적으로 단일 파일 레이아웃을 사용하는 [ComfyUI](https://github.com/comfyanonymous/ComfyUI) 또는 [Automatic1111](https://github.com/AUTOMATIC1111/stable-diffusion-webui)과 같은 확산 인터페이스와의 손쉬운 호환성.
2. 단일 파일 관리(다운로드 및 공유) 용이.

### DDUF

> [!WARNING]
> DDUF는 실험적인 파일 형식이며 관련 API는 향후 변경될 수 있습니다.

DDUF(**D**DUF **D**iffusion **U**nified **F**ormat)는 확산 모델을 저장, 배포 및 사용하는 것을 훨씬 쉽게 만들기 위해 설계된 파일 형식입니다. ZIP 파일 형식을 기반으로 하는 DDUF는 확산 모델의 모든 부분을 단일하고 관리하기 쉬운 파일로 패키징하는 표준화되고 효율적이며 유연한 방법을 제공합니다. Diffusers 다중 폴더 형식과 널리 사용되는 단일 파일 형식 간의 균형을 제공합니다.

DDUF에 대한 자세한 내용은 Hugging Face Hub [설명서](https://huggingface.co/docs/hub/dduf)를 참조하십시오.

[DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)에서 로드하려면 `dduf_file` 매개변수에 체크포인트를 전달합니다.

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "DDUF/FLUX.1-dev-DDUF", dduf_file="FLUX.1-dev.dduf", torch_dtype=torch.bfloat16
).to("cuda")
image = pipe(
    "photo a cat holding a sign that says Diffusers", num_inference_steps=50, guidance_scale=3.5
).images[0]
image.save("cat.png")

파이프라인을 `.dduf` 체크포인트로 저장하려면 필요한 모든 파일 수준 유효성 검사를 처리하는 [export_folder_as_dduf](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/serialization#huggingface_hub.export_folder_as_dduf) 유틸리티를 사용합니다.

In [ ]:
from huggingface_hub import export_folder_as_dduf
from diffusers import DiffusionPipeline
import torch 

pipe = DiffusionPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)

save_folder = "flux-dev"
pipe.save_pretrained("flux-dev")
export_folder_as_dduf("flux-dev.dduf", folder_path=save_folder)

> [!TIP]
> DDUF 형식으로 양자화된 체크포인트를 패키징하고 로드하는 것은 다중 폴더 구조를 따르는 한 지원됩니다.

## 레이아웃 및 파일 변환

Diffusers는 확산 생태계 전반에 걸쳐 더 광범위한 지원을 가능하게 하기 위해 저장소 레이아웃과 파일 형식을 변환하는 많은 스크립트와 메서드를 제공합니다.

변환 요구 사항에 맞는 스크립트를 찾으려면 [diffusers/scripts](https://github.com/huggingface/diffusers/tree/main/scripts) 컬렉션을 살펴보십시오.

> [!TIP]
> 끝에 "`to_diffusers`"가 추가된 스크립트는 모델을 Diffusers-다중 폴더 레이아웃으로 변환한다는 의미입니다. 각 스크립트에는 변환을 구성하기 위한 고유한 인수 집합이 있으므로 사용 가능한 인수를 확인하십시오!

예를 들어, Diffusers-다중 폴더 레이아웃에 저장된 Stable Diffusion XL 모델을 단일 파일 레이아웃으로 변환하려면 [convert_diffusers_to_original_sdxl.py](https://github.com/huggingface/diffusers/blob/main/scripts/convert_diffusers_to_original_sdxl.py) 스크립트를 실행합니다. 변환할 모델의 경로와 변환된 모델을 저장할 경로를 제공합니다. 선택적으로 모델을 safetensors 파일로 저장할지 여부와 모델을 반정밀도로 저장할지 여부를 지정할 수 있습니다.

```bash
python convert_diffusers_to_original_sdxl.py --model_path path/to/model/to/convert --checkpoint_path path/to/save/model/to --use_safetensors

[save_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.save_pretrained) 메서드를 사용하여 모델을 Diffusers-다중 폴더 레이아웃으로 저장할 수도 있습니다. 이렇게 하면 아직 존재하지 않는 경우 디렉터리가 생성되고 기본적으로 파일이 safetensors 파일로 저장됩니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline

pipeline = StableDiffusionXLPipeline.from_single_file(
    "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0.safetensors",
)
pipeline.save_pretrained()

마지막으로 [SD To Diffusers](https://hf.co/spaces/diffusers/sd-to-diffusers) 및 [SD-XL To Diffusers](https://hf.co/spaces/diffusers/sdxl-to-diffusers)와 같이 모델을 Diffusers-다중 폴더 레이아웃으로 변환하기 위한 보다 사용자 친화적인 인터페이스를 제공하는 공간도 있습니다. 이는 레이아웃을 변환하는 가장 쉽고 편리한 옵션이며 모델 리포지토리에 변환된 파일이 포함된 PR을 엽니다. 그러나 이 옵션은 스크립트를 실행하는 것만큼 신뢰할 수 없으며 더 복잡한 모델의 경우 공간이 실패할 수 있습니다.

## 단일 파일 레이아웃 사용법

이제 Diffusers-다중 폴더와 단일 파일 레이아웃의 차이점에 익숙해졌으므로 이 섹션에서는 [from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드를 사용하여 모델 및 파이프라인 구성 요소를 로드하고, 로드에 대한 구성 옵션을 사용자 지정하고, 로컬 파일을 로드하는 방법을 보여줍니다.

### 파이프라인 또는 모델 로드

파이프라인 또는 모델의 파일 경로를 [from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드에 전달하여 로드합니다.

<hfoptions id="pipeline-model">
<hfoption id="pipeline">

In [ ]:
from diffusers import StableDiffusionXLPipeline

ckpt_path = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0_0.9vae.safetensors"
pipeline = StableDiffusionXLPipeline.from_single_file(ckpt_path)

</hfoption>
<hfoption id="model">

In [ ]:
from diffusers import StableCascadeUNet

ckpt_path = "https://huggingface.co/stabilityai/stable-cascade/blob/main/stage_b_lite.safetensors"
model = StableCascadeUNet.from_single_file(ckpt_path)

</hfoption>
</hfoptions>

[from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드에 직접 전달하여 파이프라인의 구성 요소를 사용자 지정합니다. 예를 들어 파이프라인에서 다른 스케줄러를 사용할 수 있습니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline, DDIMScheduler

ckpt_path = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0_0.9vae.safetensors"
scheduler = DDIMScheduler()
pipeline = StableDiffusionXLPipeline.from_single_file(ckpt_path, scheduler=scheduler)

또는 파이프라인에서 ControlNet 모델을 사용할 수 있습니다.

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel

ckpt_path = "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/blob/main/v1-5-pruned-emaonly.safetensors"
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_canny")
pipeline = StableDiffusionControlNetPipeline.from_single_file(ckpt_path, controlnet=controlnet)

### 구성 옵션 사용자 지정

모델에는 UNet의 입력 수와 같은 속성을 정의하는 구성 파일이 있습니다. 파이프라인 구성 옵션은 파이프라인 클래스에서 사용할 수 있습니다. 예를 들어 [StableDiffusionXLInstructPix2PixPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/pix2pix#diffusers.StableDiffusionXLInstructPix2PixPipeline) 클래스를 보면 `is_cosxl_edit` 매개변수를 사용하여 이미지 잠재 변수를 조정하는 옵션이 있습니다.

이러한 구성 파일은 모델 Hub 리포지토리 또는 구성 파일이 시작된 다른 위치(예: GitHub 리포지토리 또는 장치의 로컬)에서 찾을 수 있습니다.

<hfoptions id="config-file">
<hfoption id="Hub configuration file">

> [!TIP]
> [from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드는 체크포인트를 적절한 모델 리포지토리에 자동으로 매핑하지만 `config` 매개변수를 사용하는 것이 유용한 경우가 있습니다. 예를 들어 체크포인트의 모델 구성 요소가 원래 체크포인트와 다르거나 체크포인트에 파이프라인에 사용할 구성을 올바르게 결정하는 데 필요한 메타데이터가 없는 경우입니다.

[from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드는 모델 리포지토리의 구성 파일에서 사용할 구성을 자동으로 결정합니다. `config` 매개변수에 리포지토리 ID를 제공하여 사용할 구성을 명시적으로 지정할 수도 있습니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline

ckpt_path = "https://huggingface.co/segmind/SSD-1B/blob/main/SSD-1B.safetensors"
repo_id = "segmind/SSD-1B"

pipeline = StableDiffusionXLPipeline.from_single_file(ckpt_path, config=repo_id)

모델은 리포지토리의 각 하위 폴더에서 [UNet](https://huggingface.co/segmind/SSD-1B/blob/main/unet/config.json), [VAE](https://huggingface.co/segmind/SSD-1B/blob/main/vae/config.json) 및 [텍스트 인코더](https://huggingface.co/segmind/SSD-1B/blob/main/text_encoder/config.json)에 대한 구성 파일을 로드합니다.

</hfoption>
<hfoption id="original configuration file">

[from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드는 다른 곳에 저장된 파이프라인의 원래 구성 파일도 로드할 수 있습니다. 원래 구성 파일의 로컬 경로 또는 URL을 `original_config` 매개변수에 전달합니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline

ckpt_path = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0_0.9vae.safetensors"
original_config = "https://raw.githubusercontent.com/Stability-AI/generative-models/main/configs/inference/sd_xl_base.yaml"

pipeline = StableDiffusionXLPipeline.from_single_file(ckpt_path, original_config=original_config)

> [!TIP]
> Diffusers는 `local_files_only=True`와 함께 `original_config`를 사용할 때 Hub의 모델 리포지토리에서 구성 파일을 가져오는 대신 파이프라인 클래스의 유형 서명을 기반으로 파이프라인 구성 요소를 추론하려고 시도합니다. 이렇게 하면 인터넷에 연결하여 필요한 구성 파일을 가져올 수 없는 코드에서 이전 버전과의 호환성이 손상되는 변경을 방지할 수 있습니다.
>
> 이는 `config` 매개변수를 사용하여 로컬 모델 리포지토리 경로를 제공하는 것만큼 신뢰할 수 없으며 파이프라인 구성 중에 오류가 발생할 수 있습니다. 오류를 방지하려면 로컬 캐시에 적절한 파이프라인 구성 파일을 다운로드하기 위해 한 번 `local_files_only=False`로 파이프라인을 실행하십시오.

</hfoption>
</hfoptions>

구성 파일은 파이프라인 또는 모델의 기본 매개변수를 지정하지만 [from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드에 매개변수를 직접 제공하여 재정의할 수 있습니다. 모델 또는 파이프라인 클래스에서 지원하는 모든 매개변수를 이 방식으로 구성할 수 있습니다.

<hfoptions id="override">
<hfoption id="pipeline">

예를 들어 [StableDiffusionXLInstructPix2PixPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/pix2pix#diffusers.StableDiffusionXLInstructPix2PixPipeline)에서 이미지 잠재 변수를 조정하려면 `is_cosxl_edit` 매개변수를 전달합니다.

In [ ]:
from diffusers import StableDiffusionXLInstructPix2PixPipeline

ckpt_path = "https://huggingface.co/stabilityai/cosxl/blob/main/cosxl_edit.safetensors"
pipeline = StableDiffusionXLInstructPix2PixPipeline.from_single_file(ckpt_path, config="diffusers/sdxl-instructpix2pix-768", is_cosxl_edit=True)

</hfoption>
<hfoption id="model">

예를 들어 [UNet2DConditionModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d-cond#diffusers.UNet2DConditionModel)에서 어텐션 차원을 업캐스트하려면 `upcast_attention` 매개변수를 전달합니다.

In [ ]:
from diffusers import UNet2DConditionModel

ckpt_path = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/blob/main/sd_xl_base_1.0_0.9vae.safetensors"
model = UNet2DConditionModel.from_single_file(ckpt_path, upcast_attention=True)

</hfoption>
</hfoptions>

### 로컬 파일

Diffusers>=v0.28.0에서 [from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드는 체크포인트 파일의 키에서 모델 유형을 추론하여 파이프라인 또는 모델을 구성하려고 시도합니다. 추론된 모델 유형은 모델 또는 파이프라인을 구성하기 위해 Hugging Face Hub에서 적절한 모델 리포지토리를 결정하는 데 사용됩니다.

예를 들어 Stable Diffusion XL 기본 모델을 기반으로 하는 모든 단일 파일 체크포인트는 [stabilityai/stable-diffusion-xl-base-1.0](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0) 모델 리포지토리를 사용하여 파이프라인을 구성합니다.

그러나 인터넷 액세스가 제한된 환경에서 작업하는 경우 [snapshot_download](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/file_download#huggingface_hub.snapshot_download) 함수를 사용하여 구성 파일을 다운로드하고 [hf_hub_download](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/file_download#huggingface_hub.hf_hub_download) 함수를 사용하여 모델 체크포인트를 다운로드해야 합니다. 기본적으로 이러한 파일은 Hugging Face Hub [캐시 디렉터리](https://huggingface.co/docs/huggingface_hub/en/guides/manage-cache)에 다운로드되지만 `local_dir` 매개변수를 사용하여 파일을 다운로드할 기본 디렉터리를 지정할 수 있습니다.

구성 및 체크포인트 경로를 [from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드에 전달하여 로컬로 로드합니다.

<hfoptions id="local">
<hfoption id="Hub cache directory">

In [ ]:
from huggingface_hub import hf_hub_download, snapshot_download

my_local_checkpoint_path = hf_hub_download(
    repo_id="segmind/SSD-1B",
    filename="SSD-1B.safetensors"
)

my_local_config_path = snapshot_download(
    repo_id="segmind/SSD-1B",
    allow_patterns=["*.json", "**/*.json", "*.txt", "**/*.txt"]
)

pipeline = StableDiffusionXLPipeline.from_single_file(my_local_checkpoint_path, config=my_local_config_path, local_files_only=True)

</hfoption>
<hfoption id="specific local directory">

In [ ]:
from huggingface_hub import hf_hub_download, snapshot_download

my_local_checkpoint_path = hf_hub_download(
    repo_id="segmind/SSD-1B",
    filename="SSD-1B.safetensors"
    local_dir="my_local_checkpoints"
)

my_local_config_path = snapshot_download(
    repo_id="segmind/SSD-1B",
    allow_patterns=["*.json", "**/*.json", "*.txt", "**/*.txt"]
    local_dir="my_local_config"
)

pipeline = StableDiffusionXLPipeline.from_single_file(my_local_checkpoint_path, config=my_local_config_path, local_files_only=True)

</hfoption>
</hfoptions>

#### 심볼릭 링크 없는 로컬 파일

> [!TIP]
> huggingface_hub>=v0.23.0에서는 [hf_hub_download](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/file_download#huggingface_hub.hf_hub_download) 및 [snapshot_download](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/file_download#huggingface_hub.snapshot_download) 함수에 `local_dir_use_symlinks` 인수가 필요하지 않습니다.

[from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드는 [huggingface_hub](https://hf.co/docs/huggingface_hub/index) 캐싱 메커니즘을 사용하여 모델 및 파이프라인에 대한 체크포인트 및 구성 파일을 가져오고 저장합니다. 심볼릭 링크를 지원하지 않는 파일 시스템에서 작업하는 경우 먼저 체크포인트 파일을 로컬 디렉터리로 다운로드하고 [hf_hub_download](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/file_download#huggingface_hub.hf_hub_download) 함수 및 [snapshot_download](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/file_download#huggingface_hub.snapshot_download) 함수에서 `local_dir_use_symlink=False` 매개변수를 사용하여 심볼릭 링크를 비활성화해야 합니다.

In [ ]:
from huggingface_hub import hf_hub_download, snapshot_download

my_local_checkpoint_path = hf_hub_download(
    repo_id="segmind/SSD-1B",
    filename="SSD-1B.safetensors"
    local_dir="my_local_checkpoints",
    local_dir_use_symlinks=False
)
print("My local checkpoint: ", my_local_checkpoint_path)

my_local_config_path = snapshot_download(
    repo_id="segmind/SSD-1B",
    allow_patterns=["*.json", "**/*.json", "*.txt", "**/*.txt"]
    local_dir_use_symlinks=False,
)
print("My local config: ", my_local_config_path)

그런 다음 로컬 경로를 `pretrained_model_link_or_path` 및 `config` 매개변수에 전달할 수 있습니다.

In [ ]:
pipeline = StableDiffusionXLPipeline.from_single_file(my_local_checkpoint_path, config=my_local_config_path, local_files_only=True)